In [1]:
pip install datasets pandas

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import pandas as pd
import wave
from datasets import load_dataset

In [4]:
class DatasetCombiner:
    def __init__(self):
        # self.dataset_10_seconds = load_dataset("pourmand1376/asr-farsi-youtube-chunked-10-seconds",split='val')
        # self.dataset_30_seconds = load_dataset("pourmand1376/asr-farsi-youtube-chunked-30-seconds")
        self.Shemo_path = '/content/drive/MyDrive/persian_datasets/Shemo'
        self.Tehran_Aghrabe_path = '/content/drive/MyDrive/persian_datasets/Tehran_Aghrabe_79'
        self.Persian_Speech_Dataset_path = '/content/drive/MyDrive/persian_datasets/persian-speech-corpus'

    # Helper function to read audio file content
    def read_audio_file(self,file_path):
        with wave.open(file_path, 'rb') as wave_file:
            n_frames = wave_file.getnframes()
            frame_rate = wave_file.getframerate()
            audio_content = wave_file.readframes(n_frames)
        return audio_content, frame_rate

    # Helper function to read transcript content
    def read_transcript_file(self,file_path):
        with open(file_path, 'r') as file:
            return file.read()

    def load_Shemo(self):
        audio_data = []
        dataset_path = self.Shemo_path
        for gender in ['female', 'male']:
          gender_path = os.path.join(dataset_path, gender)
          for file_name in os.listdir(gender_path):
              if file_name.endswith('.wav'):
                  audio_path = os.path.join(gender_path, file_name)
                  transcript_path = os.path.join(dataset_path, 'transcript','final text', file_name.replace('.wav', '.ort'))

                  if os.path.exists(transcript_path):
                      audio_content, frame_rate = self.read_audio_file(audio_path)
                      transcript_content = self.read_transcript_file(transcript_path)

                      audio_data.append({
                          # 'gender': gender,
                          'audio': audio_path,
                          'frame_rate': frame_rate,
                          'transcript': transcript_content
                      })

        df_shemo = pd.DataFrame(audio_data)
        return df_shemo

    def hf_dataset(self, dataset):
        audio_contents = []
        transcript_content = []
        frame_rates = []

        # for mode in ['train', 'test', 'val']:
        for sample in dataset:#[mode]:
            audio_contents.append(sample['audio']['array'])
            transcript_content.append(sample['transcription'])
            frame_rates.append(sample['audio']['sampling_rate'])

        df_hf_dataset = pd.DataFrame({
            'audio': audio_contents,
            'frame_rate': frame_rates,
            'transcript': transcript_content
        })

        return df_hf_dataset

    def Tehran_aghrabe(self):
        data_path = self.Tehran_Aghrabe_path
        audio_folder = os.path.join(data_path, 'audios_chunked_79')
        csv_file = os.path.join(data_path, 'Tehran_Aghrabe_79.csv')

        df = pd.read_csv(csv_file)

        audio_contents = []
        frame_rates = []
        for index, row in df.iterrows():
            audio_path = os.path.join(audio_folder, row['wav_filename'])
            audio_content, frame_rate = self.read_audio_file(audio_path)
            audio_contents.append(audio_path)
            frame_rates.append(frame_rate)

        df_Tehran_Aghrabe = pd.DataFrame({
        'audio': audio_contents,
        'frame_rate': frame_rates,
        'transcript': df['transcript']
        })

        return df_Tehran_Aghrabe


    # def load_Persian_Speech(self):
    #     audio_contents = []
    #     transcripts = []
    #     frame_rates = []
    #     valid_transcripts = []

    #     dataset_path = self.Persian_Speech_Dataset_path
    #     wav_folder = os.path.join(dataset_path, 'wav')
    #     transcript_file = os.path.join(dataset_path, 'orthographic-transcript.txt')


    #     with open(transcript_file, 'r', encoding='utf-8') as file:
    #         for line in file:
    #             parts = line.strip().split(' ', 1)
    #             filename = parts[0].strip('"')
    #             transcript = parts[1].strip('"')
    #             transcripts.append((filename, transcript))

    #     transcripts = pd.DataFrame(transcripts, columns=['filename', 'transcript'])

    #     for index, row in transcripts.iterrows():
    #         audio_path = os.path.join(wav_folder, row['filename'])
    #         if os.path.exists(audio_path):
    #             audio_content, frame_rate = self.read_audio_file(audio_path)
    #             audio_contents.append(audio_content)
    #             frame_rates.append(frame_rate)
    #             valid_transcripts.append(row['transcript'])
    #         else:
    #             print(f"File {audio_path} does not exist")

    #     df_persian_speech = pd.DataFrame({
    #     'audio': audio_contents,
    #     'frame_rate': frame_rates,
    #     'transcript': valid_transcripts
    #     })

    #     return df_persian_speech


    def combine_datasets(self):
        # df_10_seconds = self.hf_dataset(self.dataset_10_seconds)
        # df_30_seconds = self.hf_dataset(self.dataset_30_seconds)
        df_shemo = self.load_Shemo()
        df_Tehran_Aghrabe = self.Tehran_aghrabe()
        # df_persian_speech = self.load_Persian_Speech()
        print(len(df_shemo))
        print(len(df_Tehran_Aghrabe))
        print(len(df_persian_speech))

        # Combining the datasets
        combined_df = pd.concat([df_shemo, df_Tehran_Aghrabe, df_persian_speech], ignore_index=True)

        return combined_df

    def get_combined_dataset(self):
        return self.combine_datasets()



In [5]:
combiner = DatasetCombiner()
combined_dataset = combiner.get_combined_dataset()

AttributeError: 'DatasetCombiner' object has no attribute 'load_Persian_Speech'

In [1]:
print(len(combined_dataset))
print(combined_dataset.head())

NameError: name 'combined_dataset' is not defined

In [ ]:
print(combined_dataset.sample(n=5))

In [ ]:
# dataset_10_seconds = load_dataset("pourmand1376/asr-farsi-youtube-chunked-10-seconds")
# dataset_30_seconds = load_dataset("pourmand1376/asr-farsi-youtube-chunked-30-seconds")

In [ ]:
# print(type(dataset_10_seconds))
# print(dataset_10_seconds['train'][1])